

# 第5课：蒙特卡洛方法（Monte Carlo Methods）

---

## Part 1：通过例子介绍蒙特卡洛

在前几课中，我们通过 MDP 的状态转移概率 $P(s'|s,a)$ 来精确地计算值函数。但现实世界中，环境模型往往是**未知的**，这时我们可以：

> 通过**与环境交互得到的实际轨迹**，直接估计值函数 —— 这就是蒙特卡洛方法。

---

### 例子：玩 21 点（Blackjack）

我们不知道 21 点的全部概率模型，但可以让智能体玩很多局游戏，然后：

* 对每种状态 $s$，记录其在轨迹中的回报 $G_t$
* 估计状态值 $V(s)$ 为所有回报的平均值

例如，状态 `s = 18分 + 无A` 出现了 5 次，分别得到回报 0, 1, -1, 1, 1，那么估计值：

$$
V(s) = \frac{0 + 1 - 1 + 1 + 1}{5} = 0.4
$$

这就是\*\*蒙特卡洛评估（Monte Carlo Evaluation）\*\*的基本思想。

---

## Part 2：MC Basic 算法介绍（评估）

### 目标：

> 给定一个策略 $\pi$，估计它的状态值函数 $V^\pi(s)$，或动作值函数 $Q^\pi(s,a)$

---

### 核心思想：

通过执行多次 episode（完整轨迹），用**实际观测到的 return** 来计算值函数。

---

### 主要方法：

* **First-visit MC**：每次 episode 中，只对某状态第一次出现的时间步更新
* **Every-visit MC**：每次 episode 中，对该状态所有出现的时间步都更新

---

### First-Visit MC 算法伪代码（估计 $V^\pi(s)$）：

```python
Initialize: V(s) = 0, Returns(s) = []
For episode = 1 to N:
    Generate an episode: (s0, a0, r1, s1, a1, r2, ..., sT)
    G = 0
    For t = T-1 to 0:
        G ← γ * G + r_{t+1}
        If s_t is the first occurrence in episode:
            Append G to Returns(s_t)
            V(s_t) = average(Returns(s_t))
```

---

## Part 3：MC Basic 算法例子

我们用一个简单例子说明 first-visit MC 如何估计 $V(s)$：

---

### 假设：

* 状态空间：$s \in \{A, B\}$
* 策略 $\pi$ 固定，允许我们采样 episode

---

### Episode 1:

$$
(s_0 = A, a_0, r_1 = 1, s_1 = B, a_1, r_2 = 0, s_2 = Term)
$$

→ $G_0 = 1$, $G_1 = 0$

* First-visit：

  * $A$ 首次出现在 $t=0$，加入 $G=1$
  * $B$ 首次出现在 $t=1$，加入 $G=0$

---

### Episode 2:

$$
(s_0 = A, a_0, r_1 = 0, s_1 = A, a_1, r_2 = 2, s_2 = Term)
$$

→ $G_0 = 0 + 2 = 2$, $G_1 = 2$

* First-visit：

  * $A$ 首次出现在 $t=0$，加入 $G=2$

---

### 更新值：

* $V(A) = \frac{1 + 2}{2} = 1.5$
* $V(B) = \frac{0}{1} = 0$

---

## Part 4：MC Exploring Starts 算法（控制）

### 背景：

前面的 MC 只能评估**已知策略**，但我们还希望**优化策略** —— 即学出最优策略。

这就需要探索所有状态-动作对，而不是只采样固定策略。

---

### Exploring Starts 假设：

> 每个 episode 从**任意状态-动作对 $(s, a)$** 开始，并有非零概率访问所有可能的 $s, a$。

在这个假设下，我们可以收集所有 $Q(s,a)$ 的估计值，并构造最优策略。

---

### 算法结构：

```python
Initialize: Q(s,a) arbitrarily, π(s) = argmax_a Q(s,a)
Loop:
    Choose (s0, a0) with Exploring Starts
    Generate an episode following π starting with (s0, a0)
    For each (s, a) in episode:
        G = return following first occurrence of (s,a)
        Update Q(s,a) as average of returns
    Update π(s) = argmax_a Q(s,a)
```

* 策略始终对当前的 $Q(s,a)$ 贪婪
* 不断迭代更新 $Q$ 与 $\pi$

---

## Part 5：MC ε-Greedy 算法介绍（控制）

在真实任务中，“任意状态动作起始”往往难以满足，因此引入**ε-Greedy 策略探索机制**：

---

### ε-Greedy 策略：

$$
\pi(a|s) =
\begin{cases}
1 - \varepsilon + \frac{\varepsilon}{|\mathcal{A}(s)|}, & \text{if } a = \arg\max Q(s,a) \\
\frac{\varepsilon}{|\mathcal{A}(s)|}, & \text{otherwise}
\end{cases}
$$

* 以概率 $1 - \varepsilon$ 选择当前最优动作（贪婪）
* 以概率 $\varepsilon$ 随机选择其他动作（探索）

---

### 算法结构（on-policy learning）：

```python
Initialize: Q(s,a), π(s) ε-greedy w.r.t. Q
Loop:
    Generate episode using π (with ε-greedy exploration)
    For each (s,a) in episode:
        G = return following first occurrence of (s,a)
        Update Q(s,a) as average of returns
    Update π(s) = ε-greedy w.r.t. Q(s,a)
```

---

### 特点：

| 特性           | 说明                     |
| ------------ | ---------------------- |
| on-policy    | 策略使用自身数据改进自己           |
| 保证所有动作都有非零概率 | 避免陷入局部最优               |
| 易于实现         | 比 Exploring Starts 更实用 |

---

## Part 6：MC ε-Greedy 算法例子

假设一个小型迷宫任务：

* 状态 $s \in \{A, B, C\}$，动作 $a \in \{L, R\}$
* 初始 $Q(s,a) = 0$，ε = 0.1

经过多次 episode 后：

* 记录状态-动作对的所有 return 平均值更新 Q
* ε-greedy 策略选择 Q 最大的动作为主，偶尔探索

最终：

* $Q(s,a)$ 趋于真实值函数 $Q^*(s,a)$
* 策略 $\pi$ 趋于最优策略 $\pi^*$

---

## 小结

| 算法                       | 是否需要模型 | 是否用于控制  | 探索机制               |
| ------------------------ | ------ | ------- | ------------------ |
| First-Visit MC           | 否      | 否（策略评估） | 无                  |
| MC with Exploring Starts | 否      | 是（策略改进） | 每次随机起始             |
| MC with ε-Greedy         | 否      | 是（策略改进） | ε-Greedy on-policy |


